In [3]:
import os

In [4]:
pwd

'd:\\Personal\\Projects_Computer_Science_And_Engineering\\RaunakMLProjects\\End-to-end-ML-Project\\research'

In [5]:
os.chdir("../")

In [6]:
pwd

'd:\\Personal\\Projects_Computer_Science_And_Engineering\\RaunakMLProjects\\End-to-end-ML-Project'

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    # Define the fields with type annotations
    root_dir: Path
    data_path: Path

In [15]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [17]:
import os
from src.mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [24]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import logging
from imblearn.over_sampling import SMOTE

# Assuming logger is set up
logger = logging.getLogger(__name__)

class DataTransformation:
    def __init__(self, config):
        self.config = config

    ## Log Transformation Method
    def log_transform(self, column_names):
        """
        Applies log transformation to specified column(s) in the data.
        Args:
            column_names (str or list): The column(s) to apply the log transformation on.
        """
        data = pd.read_csv(self.config.data_path)

        # If a single column name is passed (string), convert it to a list
        if isinstance(column_names, str):
            column_names = [column_names]

        # Apply log transformation to each specified column
        for column_name in column_names:
            if column_name in data.columns:
                data[column_name] = np.log1p(data[column_name])  # log1p applies log(x + 1)
                logger.info(f"Applied log transformation on column: {column_name}")
            else:
                logger.warning(f"Column {column_name} not found in the dataset.")
        
        return data

    ## Outlier Removal Method
    def remove_outliers(self, df, columns):
        """
        Removes outliers from the specified columns using the IQR method.
        Args:
            df (pd.DataFrame): The dataset.
            columns (list): List of column names to check for outliers.
        """
        for column in columns:
            Q1 = df[column].quantile(0.25)  # First quartile (25th percentile)
            Q3 = df[column].quantile(0.75)  # Third quartile (75th percentile)
            IQR = Q3 - Q1                   # Interquartile range

            # Define the outlier boundaries
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Remove outliers
            df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
            logger.info(f"Outliers removed from column: {column}")

        return df

    ## Preprocessing Method that combines log transformation, outlier removal, and train-test split
    def preprocess(self):
        # Step 1: Load data
        data = pd.read_csv(self.config.data_path)

        # Step 2: Apply log transformation to specified columns
        columns_to_transform = ['BMI']  # Replace with your columns to apply log transformation
        data = self.log_transform(columns_to_transform)

        # Step 3: Remove outliers from specified columns after log transformation
        columns_to_check_for_outliers = ['BMI', 'GenHlth', 'PhysHlth', 'MentHlth']  # Replace with your columns
        data = self.remove_outliers(data, columns_to_check_for_outliers)

        # Step 4: Separate features and target variable
        target_column = 'Diabetes_012'  # Replace with your target column
        if target_column not in data.columns:
            raise ValueError(f"Target column {target_column} not found in the dataset")

        X = data.drop(columns=[target_column])
        y = data[target_column]

        # Step 3: Split into training and testing sets (before applying SMOTE)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=42, stratify=y
        )

        # Step 4: Print class frequency before resampling (in the training set)
        print("\nClass frequency in data before resampling:")
        print(y.value_counts())
        logger.info(f"Class frequency in data before resampling: {y.value_counts().to_dict()}")

        # Step 5: Apply SMOTE only on the training data
        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

        # Step 6: Print class frequency after resampling
        print("\nClass frequency in training set after resampling:")
        print(pd.Series(y_train_resampled).value_counts())
        logger.info(f"Class frequency in training set after resampling: {pd.Series(y_train_resampled).value_counts().to_dict()}")

        # Step 7: Combine features and target for saving
        train = pd.concat([X_train_resampled, y_train_resampled], axis=1)
        test = pd.concat([X_test, y_test], axis=1)

        # Step 8: Save train and test datasets
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Split data into training and test sets after applying SMOTE to the training set only")
        logger.info("SMOTE Applied Successfully:")
        logger.info(f"Train shape: {train.shape}")
        logger.info(f"Test shape: {test.shape}")

        print("\nTraining and testing datasets saved.")
        print(f"Train shape: {train.shape}")
        print(f"Test shape: {test.shape}")

In [25]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)

    # Apply log transformation outlier remove and then  train-test splitting smote
    data_transformation.preprocess()

except Exception as e:
    raise e


[2024-12-31 12:35:09,192: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-31 12:35:09,195: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-31 12:35:09,217: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-31 12:35:09,220: INFO: common: created directory at: artifacts]
[2024-12-31 12:35:09,221: INFO: common: created directory at: artifacts/data_transformation]
[2024-12-31 12:35:10,094: INFO: 1315821420: Applied log transformation on column: BMI]
[2024-12-31 12:35:10,127: INFO: 1315821420: Outliers removed from column: BMI]
[2024-12-31 12:35:10,159: INFO: 1315821420: Outliers removed from column: GenHlth]
[2024-12-31 12:35:10,187: INFO: 1315821420: Outliers removed from column: PhysHlth]
[2024-12-31 12:35:10,211: INFO: 1315821420: Outliers removed from column: MentHlth]

Class frequency in data before resampling:
Diabetes_012
0.0    147493
2.0     18626
1.0      2603
Name: count, dtype: int64
[2024-12-31 12:35:10,355: INFO: 1

TypeError: argument of type 'ConfigurationManager' is not iterable